In [26]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import imghdr
import pandas as pd

In [27]:
data_dir = '../data'
train_dir = 'train_data'
test_dir = 'test_data_v2'

In [28]:
image_exts = ['jpg', 'jpeg', 'bmp', 'png']

### check dodgy data

In [30]:
# check train dataset
for class_name in ['AI', 'human']:
    for image in os.listdir(os.path.join(data_dir, train_dir, class_name)):
        image_path = os.path.join(data_dir, train_dir, class_name, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not have a valid extension')
        except Exception as e:
            print('Issue with image {}'.format(image_path))


In [31]:
# check test dataset
for image in os.listdir(os.path.join(data_dir, test_dir)):
    image_path = os.path.join(data_dir, test_dir, image)
    try:
        img = cv2.imread(image_path)
        tip = imghdr.what(image_path)
        if tip not in image_exts:
            print('Image not have a valid extension')
    except Exception as e:
        print('Issue with image {}'.format(image_path))


### load data

In [32]:
data = tf.keras.utils.image_dataset_from_directory(os.path.join(data_dir, train_dir))

Found 79950 files belonging to 2 classes.
